In [11]:
#imports
import requests
import json
import os
import shutil

import config

DATASET_1 = 'Data_1'
DATASET_2 = 'Data_2'
CURRENT_DATASET = DATASET_1
SCENARIOS = ['Ulds_scenario_1', 'Ulds_scenario_2a', 'Ulds_scenario_2b']

PATH = './Data/' + CURRENT_DATASET
INPUT_PATH = PATH + '/2_AeJobs'
OUTPUT_PATH = PATH + '/3_AeResults'

URL = config.BACKEND_SERVER_URL

In [12]:
def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        job_json = json.load(json_file)
        return job_json

In [13]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [14]:
def createOutputFolderIfNotExists(path):
    isExist = os.path.exists(path)
    if isExist:
        shutil.rmtree(path)
        os.makedirs(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    print("The new directory is created!")

In [15]:
def read_concatenate_jobs(batch, input_path: str):
    jobs = []
    for f in batch:
        input_path_full = input_path  + '/' + f
        try:
            job = readJson(input_path_full)
            jobs.append(job)
        except:
            print("Could not read file! " + input_path_full)
            return jobs
    return jobs

In [16]:
def _store_results(r, output_path: str):
    assessment_results = r.json()
    for assessment_result in assessment_results:
        output_path_full = output_path + '/' + assessment_result["label"]
        writeJson(output_path_full, assessment_result)

In [17]:
from tqdm import tqdm

def execute_jobs_parallel(batch_size: int, input_path: str, output_path:str):
    createOutputFolderIfNotExists(output_path)
    file_names = list(fname for fname in os.listdir(input_path) if fname.endswith('.json'))

    l_index = 0
    r_index = batch_size
    current_batch = file_names[l_index:r_index]

    pbar = tqdm(total=len(file_names))

    while current_batch:
        jobs = read_concatenate_jobs(current_batch, input_path)

        r = requests.post(URL, json=jobs)

        if r.ok:
            _store_results(r, output_path)
        else:
            print("Failure " + r.text)

        l_index, r_index = r_index, r_index + batch_size
        current_batch = file_names[l_index:r_index]
        pbar.update(batch_size)
    pbar.close()

In [18]:
def executeAeJob(file_path_input, file_path_output):
    aeJob = 0
    try:
        aeJob = readJson(file_path_input)
    except:
        print("Could not read file! " + file_path_input)
        return
    
    r = requests.post(URL, json=aeJob)
    
    if r.ok:
        writeJson(file_path_output, r.json())
    else:
        print("Failure " + r.text)

In [19]:
for scenario in SCENARIOS:
    execute_jobs_parallel(40, INPUT_PATH + '/' + scenario, OUTPUT_PATH + '/' + scenario)

The new directory is created!



  9%|▉         | 120/1301 [05:00<49:15,  2.50s/it]

KeyboardInterrupt

